In this demo we will go over the basics of the Ray Job Submission Client in the SDK

In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

Here, we want to define our cluster by specifying the resources we require for our batch workload. Below, we define our cluster object (which generates a corresponding RayCluster).

NOTE: The default images used by the CodeFlare SDK for creating a RayCluster resource depend on the installed Python version:

- For Python 3.9: 'quay.io/modh/ray:2.35.0-py39-cu121'
- For Python 3.11: 'quay.io/modh/ray:2.35.0-py311-cu121'

If you prefer to use a custom Ray image that better suits your needs, you can specify it in the image field to override the default.

In [ ]:
# Create and configure our cluster object
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below
cluster = Cluster(ClusterConfiguration(
    name='jobtest',
    head_cpu_requests=1,
    head_cpu_limits=1,
    head_memory_requests=4,
    head_memory_limits=4,
    head_extended_resource_requests={'nvidia.com/gpu':1}, # For GPU enabled workloads set the head_extended_resource_requests and worker_extended_resource_requests
    worker_extended_resource_requests={'nvidia.com/gpu':1},
    num_workers=2,
    worker_cpu_requests='250m',
    worker_cpu_limits=1,
    worker_memory_requests=4,
    worker_memory_limits=4,
    # image="", # Optional Field 
    write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
    # local_queue="local-queue-name" # Specify the local queue manually
))

In [ ]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

In [ ]:
cluster.details()

### Ray Job Submission

* Initialise the Cluster Job Client 
* Provide an entrypoint command directed to your job script
* Set up your runtime environment

In [ ]:
# Initialize the Job Submission Client
"""
The SDK will automatically gather the dashboard address and authenticate using the Ray Job Submission Client
"""
client = cluster.job_client

In [ ]:
# Submit an example mnist job using the Job Submission Client
submission_id = client.submit_job(
    entrypoint="python mnist_fashion.py",
    runtime_env={"working_dir": "./","pip": "requirements.txt"},
)
print(submission_id)

In [ ]:
# Get the job's logs
client.get_job_logs(submission_id)

In [ ]:
# Get the job's status
client.get_job_status(submission_id)

In [ ]:
# Get job related info
client.get_job_info(submission_id)

In [ ]:
# List all existing jobs
client.list_jobs()

In [ ]:
# Iterate through the logs of a job 
async for lines in client.tail_job_logs(submission_id):
    print(lines, end="") 

In [ ]:
# Delete a job
# Can run client.stop_job(submission_id) first if job is still running
client.delete_job(submission_id)

In [ ]:
cluster.down()

In [ ]:
auth.logout()